In [ ]:
from bokeh.plotting import figure
from bokeh.layouts import row, column, widgetbox, gridplot

from bokeh.io import curdoc
from bokeh.io import output_notebook, show, output_file

from bokeh.models import ColumnDataSource, HoverTool, TapTool, Range1d, OpenURL
from bokeh.models import LinearColorMapper , ColorBar
from bokeh.models.widgets import Select, Slider


import numpy as np

In [ ]:
# =============================================
# THIS comes from INTERFACE
#
selected_exposure = '00000003'# args['exposure']
selected_arm = 'r' # args['arm']
selected_spectrograph =  4 # args['spectrograph']

# =============================================
# THIS comes from QLF.CFG
#
night = '20190101'

# ============================================
#  THIS READ yaml files
#
from scalar_metrics_v02 import LoadMetrics

cam = selected_arm+str(selected_spectrograph)
exp = selected_exposure # intentionaly redundant
lm = LoadMetrics(cam, exp, night);
metrics, tests  = lm.metrics, lm.tests 

# =============================================
# THIS is only to simplify the code understanding
#
countpix  = metrics['countpix']
getbias   = metrics['getbias']
getrms    = metrics['getrms']
xwsigma   = metrics['xwsigma']
countbins = metrics['countbins']
integ     = metrics['integ']
skycont   = metrics['skycont']
skypeak   = metrics['skypeak']
skyresid  = metrics['skyresid']
snr       = metrics['snr']

In [ ]:
xsigma_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">XSigma: </span>
            <span style="font-size: 13px; color: #515151">@xsigma</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">RA: </span>
            <span style="font-size: 13px; color: #515151;">@x1</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">DEC: </span>
            <span style="font-size: 13px; color: #515151;">@y1</span>
        </div>
    </div>
"""

wsigma_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">WSigma: </span>
            <span style="font-size: 13px; color: #515151">@wsigma</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">RA: </span>
            <span style="font-size: 13px; color: #515151;">@x1</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">DEC: </span>
            <span style="font-size: 13px; color: #515151;">@y1</span>
        </div>
    </div>
"""



# determining the position of selected cam fibers:
c1,c2 = int(selected_spectrograph)*500, (int(selected_spectrograph)+1)*500
qlf_fiberid = np.arange(0,5000)[c1:c2] 

xsigma_hover = HoverTool(tooltips=xsigma_tooltip)
wsigma_hover = HoverTool(tooltips=wsigma_tooltip)

xsigma = xwsigma['XSIGMA']
wsigma = xwsigma['WSIGMA']

source = ColumnDataSource(data={
    'x1'     : xwsigma['RA'][c1:c2],
    'y1'     : xwsigma['DEC'][c1:c2],
    'xsigma' : xwsigma['XSIGMA'],
    'wsigma' : xwsigma['WSIGMA'],
    'QLF_FIBERID': qlf_fiberid,
})

source_comp = ColumnDataSource(
    data = {
    'x1': xwsigma['RA'][:c1] + xwsigma['RA'][c2:],
    'y1': xwsigma['DEC'][:c1] + xwsigma['DEC'][c2:],
    'xsigma': ['']*4500,
    'wsigma': ['']*4500
})

# axes limit
left, right = 331, 335
bottom, top = 13, 16.7

xmapper = LinearColorMapper(palette= my_palette,
                           low=0.98*np.min(xsigma), 
                           high=1.02*np.max(xsigma))

wmapper = LinearColorMapper(palette= my_palette,
                           low=0.99*np.min(wsigma), 
                           high=1.01*np.max(wsigma))

# ======
# XSIGMA
px = figure( title = 'XSIGMA', x_axis_label='RA', y_axis_label='DEC'
           , plot_width=700, plot_height=600
           , x_range=Range1d(left, right), y_range=Range1d(bottom, top)
           , tools= [xsigma_hover, "pan,box_zoom,reset,crosshair"])

# Color Map
px.circle('x1','y1', source = source, name="data", radius = 0.018,
        fill_color={'field': 'xsigma', 'transform': xmapper}, 
         line_color='black', line_width=0.1,
         hover_line_color='red')

# marking the Hover point
px.circle('x1','y1', source = source, name="data", radius = 0.0186
          , fill_color=None, line_color=None
          , line_width=3, hover_line_color='red')

px.circle('x1','y1', source = source_comp, radius = 0.015,
         fill_color = 'lightgray', line_color='black', line_width=0.3)

# bokeh.pydata.org/en/latest/docs/reference/models/annotations.html
xcolor_bar = ColorBar(color_mapper= xmapper, label_standoff=-13,
                     major_label_text_font_style="bold", padding = 26,
                     major_label_text_align='right',
                     major_label_text_font_size="10pt",
                     location=(0, 0))

px.add_layout(xcolor_bar, 'left')



# ======
# WSIGMA
pw = figure( title = 'WSIGMA', x_axis_label='RA', y_axis_label='DEC'
           , plot_width=700, plot_height=600
           , x_range=Range1d(left, right), y_range=Range1d(bottom, top)
           , tools= [wsigma_hover, "pan,box_zoom,reset,crosshair"])

# Color Map
pw.circle('x1','y1', source = source, name="data", radius = 0.018,
        fill_color={'field': 'wsigma', 'transform': wmapper}, 
         line_color='black', line_width=0.1,
         hover_line_color='red')

# marking the Hover point
pw.circle('x1','y1', source = source, name="data", radius = 0.0186
          , fill_color=None, line_color=None
          , line_width=3, hover_line_color='red')

pw.circle('x1','y1', source = source_comp, radius = 0.015,
         fill_color = 'lightgray', line_color='black', line_width=0.3)

# bokeh.pydata.org/en/latest/docs/reference/models/annotations.html
wcolor_bar = ColorBar(color_mapper= wmapper, label_standoff=-13,
                     major_label_text_font_style="bold", padding = 26,
                     major_label_text_align='right',
                     major_label_text_font_size="10pt",
                     location=(0, 0))

pw.add_layout(wcolor_bar, 'left')

layout= column([px,pw])#, sizing_mode='scale_width')
#output_file('out_xwsigma.html')
output_notebook()
show(layout)  

In [10]:
hist_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">Frequency: </span>
            <span style="font-size: 13px; color: #515151">@hist</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">sigma: </span>
            <span style="font-size: 13px; color: #515151;">[@left, @right]</span>
        </div>
    </div>
"""
sname = 'XSIGMA'

def bins_dane(data):
    # Dane's rule of thumb for bins
    ndata = len(data)
    mean = np.mean(data)
    sigma = np.std(data)
    b = sum([ (x - mean)**3 for x in xwsigma[sname]])
    b = b/sum([ (x - mean)**2 for x in xwsigma[sname]])**(1.5)
    return int(round(np.log2(ndata) + 1 + np.log2((1.+b)/(sigma*b))))

hover = HoverTool(tooltips=hist_tooltip)
bins = bins_dane(xwsigma[sname]) #17
hist, edges = np.histogram(xwsigma[sname], bins = bins)

source = ColumnDataSource(data={
    'hist':hist,
    'bottom':[0] *len(hist),
    'left':edges[:-1],
    'right':edges[1:]
})

p = figure(title=sname,tools=[hover,"pan,wheel_zoom,box_zoom,reset"],
           y_axis_label='Frequency', x_axis_label=sname,
           #y_axis_type='log',
            background_fill_color="white")

p.quad(top='hist', bottom='bottom', left='left', right='right',
       source=source, 
        fill_color="dodgerblue", line_color="black", alpha=0.8,
       hover_fill_color='blue', hover_line_color='black', hover_alpha=0.8)
p.legend.location = "top_left"
output_notebook()
show(p)

Loading BokehJS ...

In [ ]:
def bins_dane(data):
    # Dane's rule of thumb for bins
    ndata = len(data)
    mean = np.mean(data)
    sigma = np.std(data)
    b = sum([ (x - mean)**3 for x in xwsigma['XSIGMA']])
    b = b/sum([ (x - mean)**2 for x in xwsigma['XSIGMA']])**(1.5)
    return int(round(np.log2(ndata) + 1 + np.log2((1.+b)/(sigma*b))))
    
ndata = len(xwsigma['XSIGMA'])
mean = np.mean(xwsigma['XSIGMA'])
sigma = np.std(xwsigma['XSIGMA'])

b = sum([ (x - mean)**3 for x in xwsigma['XSIGMA']])
b = b/sum([ (x - mean)**2 for x in xwsigma['XSIGMA']])**(1.5)
danesrule= int(round(np.log2(ndata) + 1 + np.log2((1.+b)/(sigma*b))))
sturgesrule = int(round(1.+3.322 *np.log10(500) ))
scottsrule= int(round(3.49 *sigma/(ndata)**(1./3.)))

In [ ]:
1 + 3.322 *np.log10(55) #= 6.781 ≅ 7

In [ ]:
print(danesrule, sturgesrule, scottsrule)